In [ ]:
import pandas as pd

from datamender import Mender

# main_df = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/nba.csv")
main_df = pd.read_csv('./expermental/testing_data/titanic.csv')

main_mender = Mender(main_df)

In [ ]:
# Show all the columns
main_df.head()

In [ ]:
main_mender.overview()

In [ ]:
main_mender.insights()

In [ ]:
main_mender.show_incomplete(50)

## Imputation Plan

Sex is object, but need to be converted to number or boolean, can use one-hot
Embarked can also be converted with one-hot

Age and cabin have most of missing value, should create a new column - Age__missing to capture the missing value in age

Cabin is too complex to process, need to ignore



In [ ]:
main_mender.df.head()

In [ ]:
# main_mender.setfocus(columns=['Survived', 'Pclass', 'Name'], rows=(0, 50))

In [ ]:
# Ignore Cabin colum because it is string value without any pattern
main_mender.setignore(columns=['Cabin'])

In [ ]:
from datamender import ImputationStrategies as im_strategy

# Apply mean imputation to the age column
# But because the age column contains too much missing data, we'll capture the likelihood of missing as new column
main_mender.impute_columns(
    columns=["Age"],
    strategies=[im_strategy.mean],
    inplace= True,
    capture_likelihood = True
)


In [ ]:
# Since there are only 2 value missed in this column
# Impute with mode since the majority takes 72.28% of the value
main_mender.impute_columns(
    columns=["Embarked"],
    strategies=[im_strategy.mode],
    inplace=True
)

In [ ]:
from datamender import EncodingStrategies as en_stragety

# Sex is object, but need to be converted to number or boolean, can use one-hot
main_mender.encode_column('Sex', en_stragety.one_hot)

In [ ]:
#Embarked can also be converted with one-hot
main_mender.encode_column('Embarked', en_stragety.one_hot)

In [ ]:
import re
def amend_height(feet):
    pattern = re.compile(r"(\d+)-(\d+)")
    if pd.isna(feet):
        return None

    found = pattern.findall(feet)
    if len(found) == 0:
        return None

    feet, inch = (found[0])
    return int(feet) * 30.48 + int(inch) * 2.54

main_mender.amend_column("Height", amend_height)

In [ ]:
main_mender.graphs(kind="kde")

In [ ]:

main_mender.setfocus(columns=['Survived'], rows=(1, 50, 2))

In [ ]:
main_mender.column_categories("Position")

In [ ]:
from datamender import EncodingStrategies as en_strategy

category_order_map = {
    "PG": 1,
    "SF": 3,
    "SG": 5,
    "PF": 7,
    "C": 100,
}

main_mender.encode_column(
    column="Position",
    strategy=en_strategy.label_encoding,
    category_order_map=category_order_map
)
